In [ ]:
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# os
import os




In [ ]:
# Step 1: Load Images and Labels
def load_images(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

folder = "train_data"
images = load_images(folder)
labels = pd.read_csv("train_data/train_data_labels.csv", header=None).values.ravel()



In [ ]:
# Step 2: Preprocess Images
def preprocess_images(images):
    processed_images = [cv2.resize(img, (200, 200)).flatten() for img in images]
    return np.array(processed_images) / 255.0  # Normalize to [0, 1]

processed_images = preprocess_images(images)



In [ ]:
# Step 3: Split Data
X_train, X_test, y_train, y_test = train_test_split(processed_images, labels, test_size=0.2, random_state=42)



In [ ]:
# Step 4: Train KNN Classifier
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)



In [ ]:
# Step 5: Evaluate KNN Classifier
y_pred = knn.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f'Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}')
print(f'Classification Report:\n{classification_report(y_test, y_pred)}')

# P approach

In [13]:
import torch
import torch.optim as optim
from torchvision import datasets, models, transforms
import os
import csv
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image

# Load the pre-trained model (e.g., ResNet)
model_up = models.resnet18(pretrained=True)

# Freeze all the layers in the model
for param in model_up.parameters():
    param.requires_grad = False

# Replace the final layer of the model to match the number of classes in the new dataset
num_ftrs = model_up.fc.in_features
model_up.fc = torch.nn.Linear(num_ftrs, 2)  # num_classes should be the number of classes in your new dataset

# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model_up.fc.parameters(), lr=0.001, momentum=0.9)

class CustomDataset(Dataset):
    def __init__(self, image_dir, labels_csv, transform=None):
        self.image_dir = image_dir
        # Filter out non-image files from the directory
        self.image_files = sorted([f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif'))])
        
        # Using csv.reader to read labels from csv file
        with open(labels_csv, mode='r') as csv_file:
            csv_reader = csv.reader(csv_file)
            self.labels = [int(row[0]) for row in csv_reader]

        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.image_files[idx])
        image = Image.open(img_name)
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


# Define transformations for the images
train_transforms_up = transforms.Compose([
    transforms.Resize((100, 100)),  # adjust size if necessary
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_data = CustomDataset(image_dir="train_data", labels_csv="train_data/train_data_labels.csv", transform=train_transforms_up)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)  # batch_size reduced to 16 to prevent memory issues

# Training the model
num_epochs = 15
for epoch in range(num_epochs):  
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model_up(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print('Epoch [%d/%d], Loss: %.4f' % (epoch+1, num_epochs, running_loss/len(train_loader)))


IndexError: list index out of range

In [12]:
# ... [previous code remains unchanged]

# Evaluate the model after training to calculate accuracy and confusion matrix
model_up.eval()  # set the model to evaluation mode

all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in train_loader:
        outputs = model_up(inputs)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.numpy().tolist())
        all_labels.extend(labels.numpy().tolist())

# Calculate accuracy
correct_preds = sum([1 for actual, pred in zip(all_labels, all_preds) if actual == pred])
accuracy = correct_preds / len(all_labels)

# Compute confusion matrix without sklearn
num_classes = 2
confusion = [[0] * num_classes for _ in range(num_classes)]
for actual, pred in zip(all_labels, all_preds):
    confusion[actual][pred] += 1

print(f'Accuracy: {accuracy * 100:.2f}%')
print('Confusion Matrix:')
for row in confusion:
    print(row)

# Testing on the test_data folder and outputting the classifications into a txt file

# Modify the CustomDataset for image-only mode (no labels)
class ImageOnlyDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.image_files = sorted([f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif'))])
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.image_files[idx])
        image = Image.open(img_name)

        if self.transform:
            image = self.transform(image)

        return image, img_name  # return image name too for reference

test_data = ImageOnlyDataset(image_dir="test_data", transform=train_transforms_up)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=16, shuffle=False)

# Output classifications to a txt file
with open("classifications.csv", "w") as f:
    with torch.no_grad():
        for inputs, image_names in test_loader:
            outputs = model_up(inputs)
            _, predicted = torch.max(outputs, 1)
            for img, pred in zip(image_names, predicted.numpy()):
                f.write(f'{pred}\n')


Accuracy: 100.00%
Confusion Matrix:
[31, 0]
[0, 69]
